In [ ]:
# Step 1: Install necessary packages
!pip install streamlit opencv-python pytesseract pillow numpy pyngrok --quiet

# Step 2: Write the Streamlit app code into a file
app_code = """
import streamlit as st
import cv2
import numpy as np
import pytesseract
from PIL import Image

def fix_ocr_errors(text):
    corrections = {
        'G': '6',
    }
    fixed_text = ''
    for char in text:
        if char in corrections:
            fixed_text += corrections[char]
        else:
            fixed_text += char
    return fixed_text

def detect_number_plate(image):
    img = np.array(image)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    img_rgb = img.copy()

    hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
    lower_yellow = np.array([15, 80, 80])
    upper_yellow = np.array([40, 255, 255])
    mask_yellow = cv2.inRange(hsv, lower_yellow, upper_yellow)
    contours, _ = cv2.findContours(mask_yellow, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    plate_img = None
    for cnt in contours:
        x, y, w, h = cv2.boundingRect(cnt)
        aspect_ratio = w / float(h)
        area = w * h
        if 2 < aspect_ratio < 6 and area > 1000:
            cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 3)
            plate_img = img_bgr[y:y+h, x:x+w]
            break

    if plate_img is not None:
        plate_gray = cv2.cvtColor(plate_img, cv2.COLOR_BGR2GRAY)
        _, plate_thresh = cv2.threshold(plate_gray, 150, 255, cv2.THRESH_BINARY)
        plate_thresh = cv2.resize(plate_thresh, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)
        custom_config = r'--oem 3 --psm 7 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789'
        extracted_text = pytesseract.image_to_string(plate_thresh, config=custom_config).strip()
        fixed_text = fix_ocr_errors(extracted_text)
    else:
        extracted_text = "Plate not detected"
        fixed_text = extracted_text

    return img_rgb, plate_img, fixed_text

st.title("Number Plate Detection & OCR")

uploaded_file = st.file_uploader("Upload a car image", type=["jpg", "png", "jpeg"])

if uploaded_file is not None:
    image = Image.open(uploaded_file)
    st.image(image, caption='Uploaded Image', use_container_width=True)

    if st.button("Detect Number Plate"):
        highlighted_img, plate_img, text = detect_number_plate(image)

        st.image(highlighted_img, caption="Plate Highlighted", use_container_width=True)

        if plate_img is not None:
            st.image(cv2.cvtColor(plate_img, cv2.COLOR_BGR2RGB), caption="Cropped Plate", use_container_width=False)
        st.success(f"Detected Plate Number: {text}")
"""

with open("app.py", "w") as f:
    f.write(app_code)

# Step 3: Run Streamlit with ngrok tunnel
from pyngrok import ngrok

# Kill existing tunnels (if any)
ngrok.kill()

# **Get your authtoken from the ngrok dashboard and paste it here:**
NGROK_AUTH_TOKEN = "2wtDkYmeHWyXLqzXz7brjVfm50P_7unnpnhVFx9yumNAs95EJ"
ngrok.set_auth_token(NGROK_AUTH_TOKEN) # Authenticate with ngrok

# Set up a tunnel
public_url = ngrok.connect(8501)
print(f"Your Streamlit app is live at: {public_url}")

# Step 4: Run the app
!streamlit run app.py &

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 57.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 86.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.9 MB/s eta 0:00:00
Your Streamlit app is live at: NgrokTunnel: "https://b3a9-34-29-192-143.ngrok-free.app" -> "http://localhost:8501"



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.29.192.143:8501

